In [161]:
%%time
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import wave
import numpy
from music21 import converter, instrument, note, chord
import glob
import os
import Paths
from pydub import AudioSegment
from keras.models import Sequential
from keras.layers import Dense, LSTM, MaxPooling2D, Dropout, Activation
import librosa
import tensorflow as tf

directory = os.listdir(Paths.IN_DIRECTORY)
array = []

sequence_length = []
for sample_name in directory:
    if 'wav' in sample_name:
        file_path = os.path.join(Paths.IN_DIRECTORY, sample_name)
        song, sr = librosa.load(file_path)
        sequence_length.append(len(song))
        #print(len(song))
        #song = AudioSegment.from_wav(file_path)
        #samples = song.get_array_of_samples()
        array.append(song)

# Removing the last element using slicing
lastElementIndex = len(array)-1
in_array = array[:lastElementIndex]
out_array = array[1:]
'''
# Create flat list of Input and Output
Input_flat=[]
for sublist in In_array:
    for element in sublist:
        Input_flat.append(element)

Output_flat=[]
for sublist in Out_array:
    for element in sublist:
        Output_flat.append(element)


# create output sequence for samples following input
output = []
for i in range(0, len(array)-1):
 i+=1
 #print(i)
 output.append(array[i])
'''

#Create the Input and Output to tensors
input_tensor = tf.convert_to_tensor(in_array)
output_tensor = tf.convert_to_tensor(out_array)


batch_size = len(input_tensor)
timestep = sequence_length[0]

input_data = tf.reshape(input_tensor, shape=(batch_size, timestep, 1))
output_data = tf.reshape(output_tensor, shape=(batch_size, timestep, 1))


model = Sequential()
model.add(LSTM(units= 2, batch_input_shape= [9, 66150, 1], stateful=True, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(32, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(32))
#model.add(Dense(32))
#model.add(Dropout(0.3))
#model.add(Dense(n_vocab))
#model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(input_data, output_data, epochs=200, batch_size=sequence_length[0], callbacks=callbacks_list)


Epoch 1/200


ValueError: in user code:

    File "C:\Users\hennm\PycharmProjects\ebeat\venv\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hennm\PycharmProjects\ebeat\venv\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hennm\PycharmProjects\ebeat\venv\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\hennm\PycharmProjects\ebeat\venv\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\hennm\PycharmProjects\ebeat\venv\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\hennm\PycharmProjects\ebeat\venv\lib\site-packages\keras\engine\compile_u

In [152]:

print(input_data.shape)


(9, 66150, 1)


In [ ]:
'''

 sequence_in = flatlist[i:i + sequence_length[i]]
    sequence_out = flatlist[i + sequence_length[i]]
    network_input.append(sequence_in)
    network_output.append(sequence_out)
n_patterns = len(network_input)

# sequence_length = len(song)
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in flatlist))

n_vocab = len(set(flatlist))

# create a dictionary to map pitches to integers
note_to_int = dict((flatlist, number) for number, flatlist in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(flatlist) - sequence_length, 1):
    sequence_in = flatlist[i:i + sequence_length]
    sequence_out = flatlist[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)


# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

'''

In [48]:
#print(network_input)
print(array)


[array([0.00456181, 0.00744834, 0.00696913, ..., 0.03612468, 0.04514919,
       0.        ], dtype=float32), array([0.04360344, 0.06247874, 0.05309957, ..., 0.12486256, 0.12134458,
       0.1312821 ], dtype=float32), array([-0.01130085, -0.0170629 , -0.01534307, ..., -0.02985282,
       -0.0294109 , -0.03418307], dtype=float32), array([0.07419518, 0.10791647, 0.0935066 , ..., 0.01373434, 0.01630254,
       0.01927015], dtype=float32), array([-0.00057893, -0.00129468, -0.00135118, ..., -0.01974932,
       -0.02698674,  0.        ], dtype=float32)]


In [52]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights-improvement-59-2.5657-bigger.hdf5')


ValueError: Cannot assign value to variable ' lstm_33/lstm_cell_33/kernel:0': Shape mismatch.The variable shape (1, 2048), and the assigned value shape (1, 1024) are incompatible.

In [33]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

506


In [57]:
network_input.shape[1]

100

In [58]:
network_input.shape[2]

1

In [54]:
network_output.shape

(1670, 1521)

In [1]:
print(pitchnames)

NameError: name 'pitchnames' is not defined